In [ ]:
!python3 -m pip install --upgrade pip
!pip install -q transformers
!pip install peft
!pip install bitsandbytes
!pip install accelerate
!pip install datasets

In [1]:
!pip install peft
!pip install datasets

In [2]:
import os
import random
import string
import torch
import transformers
from peft import LoraConfig, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from datasets import Dataset

In [8]:
# loading model and the tokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [5]:
input_text = "translate English to German: How old are you?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

<pad> Wie alte sind Sie?</s>


In [6]:
input_text = "Write an SQL query to calculate the total salary expenditure for all employees in the 'employees' table."
input_ids = tokenizer(input_text, return_tensors="pt").input_ids


outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

<pad> t = t.salary('<unk>') t = t


Text Generation with Prompt engineering

zero shot inference

In [39]:
topic = 'lost time traveller'

prompt = f"""
Prompt:
Write a story about {topic}

Story:
    """

inputs = tokenizer(prompt, return_tensors='pt')

In [40]:
output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=75,
        )[0],
        skip_special_tokens=True
    )

dash_line = '-'.join('' for x in range(100))

print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}\n')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Prompt:
Write a story about lost time traveller

Story:
    
---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
The story begins in the year 2021. A man named John is a time traveller. He travels back in time to the year 2021. He travels back to the year 2021. He travels back to the year 2021. He travels back to the year 2021. He travels back to the year 2021. He travels



One shot Inference

In [41]:
topic = 'A lost sea sailor stuck at a remote island'

prompt = f"""
Prompt:
Write a story about a lost time traveler.

Story:
The time traveler stumbled out of the portal into an unfamiliar landscape. Confused and disoriented, he realized that he had lost his way in time. Now, he must find a way back to his own era before he becomes lost in the currents of time forever.

Prompt:
Write a story about {topic}

Story:

    """

inputs = tokenizer(prompt, return_tensors='pt')

In [42]:
output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=75,
        )[0],
        skip_special_tokens=True
    )

dash_line = '-'.join('' for x in range(100))

print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}\n')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Prompt:
Write a story about a lost time traveler.

Story:
The time traveler stumbled out of the portal into an unfamiliar landscape. Confused and disoriented, he realized that he had lost his way in time. Now, he must find a way back to his own era before he becomes lost in the currents of time forever.

Prompt:
Write a story about A lost sea sailor stuck at a remote island

Story:

    
---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
The sea sailor was lost at sea. He was stranded on a remote island. He was unable to find a way back to shore. He decided to stay on the island.



Few shot Inference

In [45]:
topic = 'A lost astronaut stranded on an unknown planet'

prompt = f"""
Prompt:
Write a story about a lost time traveler.

Story:
The time traveler stumbled out of the portal into an unfamiliar landscape. Confused and disoriented, he realized that he had lost his way in time. Now, he must find a way back to his own era before he becomes lost in the currents of time forever.

Prompt:
Write a story about a lost sea sailor stuck at a remote island.

Story:
The sailor's ship was wrecked in a violent storm, leaving him stranded on a desolate island in the heart of the vast ocean. With no sign of rescue in sight, he must summon all his courage and resourcefulness to survive in this harsh, unforgiving environment. Alone and facing countless challenges, he embarks on a journey of self-discovery, drawing strength from the untamed beauty of the island and the endless expanse of the sea that surrounds it.

Prompt:
Write a story about a lost hiker trapped in a mysterious forest.

Story:
A hiker loses his way in a dense forest, surrounded by towering trees and eerie silence. With no sense of direction, he faces the challenge of finding his way back to safety, navigating through shadows and unknown dangers lurking in the darkness.

Prompt:
Write a story about {topic}

Story:
    """

inputs = tokenizer(prompt, return_tensors='pt')

In [46]:
output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=75,
        )[0],
        skip_special_tokens=True
    )

dash_line = '-'.join('' for x in range(100))

print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}\n')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Prompt:
Write a story about a lost time traveler.

Story:
The time traveler stumbled out of the portal into an unfamiliar landscape. Confused and disoriented, he realized that he had lost his way in time. Now, he must find a way back to his own era before he becomes lost in the currents of time forever.

Prompt:
Write a story about a lost sea sailor stuck at a remote island.

Story:
The sailor's ship was wrecked in a violent storm, leaving him stranded on a desolate island in the heart of the vast ocean. With no sign of rescue in sight, he must summon all his courage and resourcefulness to survive in this harsh, unforgiving environment. Alone and facing countless challenges, he embarks on a journey of self-discovery, drawing strength from the untamed beauty of the island and the endless expanse of the sea that surrounds it.

Prompt:
Write a story about a lost hiker trapped